# Ch 5 - 3. 문자열 검색

문자열 검색은 마스킹 검색의 특수한 경우이다. 

시리즈에 str accessor를 사용할 수 있다. 문자열로만 구성된 시리즈에 대해서는 이를 사용할 수 있고, 이를 사용하면 string 관련 내장 함수들을 자유롭게 사용할 수 있다.  

`Series.str.strip()` : 앞 뒤 공백 제거  
`Series.str.contains(S)` : 문자열 S를 포함하고 있는지 여부를 반환  
`Series.str.split(sep, expand) ` : 구분자를 기준으로 시리즈를 분할  
> 여기서 expand = True : 새로운 열 생성  
        expand = False : 새로운 열을 생성하지 않고 리스트를 반환  

## Series.astype
자료형을 변환하는 데 사용하는 함수.  
`Series.astype(str)` : 숫자 자료형에 str accessor를 사용하기 위해 문자열로 바꿈  



## 경로 설정

In [1]:
import os
import pandas as pd

os.chdir(r"/Users/Angela/Desktop/과속대학쿠쿠루/1. 데이터 핸들링/데이터")

In [2]:
df = pd.read_excel("불량 기록 데이터.xlsx")

In [3]:
# 시리얼 번호 = 공정 - 제품 - 식별자
df.head()

,시리얼번호,불량 여부
0,S-A-001,양품
1,S-A-002,양품
2,S-A-003,양품
3,S-A-004,양품
4,S-A-005,양품


## .value_counts() 
값 별로 개수 확인하기

In [4]:
df['불량 여부'].value_counts()

양품    715
불량     41
Name: 불량 여부, dtype: int64

## 특정 글자 포함여부

In [5]:
# 공정이 S인 제품의 개수 : True의 sum()
df['시리얼번호'].str.contains('S').sum()

325

## 구분자로 분할하기

만약 분할 결과가 개수가 다를 경우, 큰 쪽에 맞춰서 열이 만들어진다.  


In [6]:
# 리스트로만 받기
df['시리얼번호'].str.split('-', expand = False).head()

0    [S, A, 001]
1    [S, A, 002]
2    [S, A, 003]
3    [S, A, 004]
4    [S, A, 005]
Name: 시리얼번호, dtype: object

In [7]:
# 하이픈 기준으로 시리얼번호 분할하기
df['시리얼번호'].str.split('-', expand = True).head()

,0,1,2
0,S,A,001
1,S,A,002
2,S,A,003
3,S,A,004
4,S,A,005


In [8]:
# 분할한 데이터를 기존 데이터에 붙이기 
merged_df = pd.concat([df, df['시리얼번호'].str.split('-', expand = True)], 
                      axis = 1) # 열방향으로 합침
merged_df

,시리얼번호,불량 여부,0,1,2
0,S-A-001,양품,S,A,001
1,S-A-002,양품,S,A,002
2,S-A-003,양품,S,A,003
3,S-A-004,양품,S,A,004
4,S-A-005,양품,S,A,005
...,...,...,...,...,...
751,T-B-237,양품,T,B,237
752,T-B-238,양품,T,B,238
753,T-B-239,양품,T,B,239
754,T-B-240,불량,T,B,240


# column 이름 변경하기

방법1 : DataFrame.columns = []  
방법2 : DataFrame.rename({기존:새이름}, axis = 1, inplace = True/False)  

전부를 새로 입힐 땐 1, 특정 컬럼만 다시 입힐 땐 2를 사용하는 것이 바람직하다.  

방법2의 경우 axis= 0이면 행 인덱스를 바꾼다고 생각하면 된다.

In [9]:
# merged_df.columns = ['A1', 'A2', 'A3', 'A4', 'A5']

In [10]:
merged_df.rename({0:'공정', 1:'제품', 2:'식별자'}, axis = 1, inplace = True)
merged_df.head()

,시리얼번호,불량 여부,공정,제품,식별자
0,S-A-001,양품,S,A,001
1,S-A-002,양품,S,A,002
2,S-A-003,양품,S,A,003
3,S-A-004,양품,S,A,004
4,S-A-005,양품,S,A,005


# pivot_table 연습하기

불량인 제품을 공정별, 제품별 개수 확인하기.



In [11]:
pd.pivot_table(merged_df.loc[merged_df['불량 여부'] != '양품'],  
               index = '공정', columns = '제품', aggfunc = 'count')

불량 여부     시리얼번호     식별자    
제품     A   B     A   B   A   B
공정                            
S      9  12     9  12   9  12
T     12   8    12   8  12   8

In [12]:
pd.pivot_table(merged_df[merged_df['불량 여부'] != '양품'], values = '불량 여부',
               index = '공정', columns = '제품', aggfunc = 'count')

제품,A,B
공정,,
S,9,12
T,12,8
